## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_clean = application_df.drop(columns = ['EIN', 'NAME'], axis = 1)

In [ ]:
# Determine the number of unique values in each column.
for n in application_df_clean.columns:
    print(n, len(application_df_clean[n].unique()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_value_count = application_df_clean["APPLICATION_TYPE"].value_counts()
application_value_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_value_count[application_value_count < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_value_count = application_df_clean["CLASSIFICATION"].value_counts()
classification_value_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_more_than_1 = classification_value_count[classification_value_count > 1]
# # Replace in dataframe
classification_more_than_1 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_to_replace = list(classification_value_count[classification_value_count < 700].index)

# Replace in dataframe
for cls in classification_to_replace:
    application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_clean['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_data_df = pd.get_dummies(application_df_clean)
dummies_data_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = dummies_data_df["IS_SUCCESSFUL"].values
X = dummies_data_df.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=hidden_nodes_layer1, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, input_dim=hidden_nodes_layer2,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 80)                3600      
                                                                 
 dense_20 (Dense)            (None, 30)                2430      
                                                                 
 dense_21 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,061
Trainable params: 6,061
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=80)

Epoch 1/80
804/804 [==============================] - 3s 2ms/step - loss: 0.5730 - accuracy: 0.7190
Epoch 2/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5597 - accuracy: 0.7245
Epoch 3/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5558 - accuracy: 0.7273
Epoch 4/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7282
Epoch 5/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accuracy: 0.7288
Epoch 6/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5510 - accuracy: 0.7298
Epoch 7/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7297
Epoch 8/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7294
Epoch 9/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7297
Epoch 10/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7313

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5504 - accuracy: 0.7380 - 517ms/epoch - 2ms/step
Loss: 0.5504325032234192, Accuracy: 0.7379592061042786


In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

In [ ]:
# Optimize the model 

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 16
hidden_nodes_layer3 = 24

nn_opt = tf.keras.models.Sequential()

# First hidden layer
nn_opt.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn_opt.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=hidden_nodes_layer1, activation='relu'))

# Third hidden layer
nn_opt.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn_opt.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_opt.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 8)                 360       
                                                                 
 dense_16 (Dense)            (None, 16)                144       
                                                                 
 dense_17 (Dense)            (None, 24)                408       
                                                                 
 dense_18 (Dense)            (None, 1)                 25        
                                                                 
Total params: 937
Trainable params: 937
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_opt.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
fit_model = nn_opt.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 6s 6ms/step - loss: 0.6015 - accuracy: 0.6927
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5681 - accuracy: 0.7218
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5609 - accuracy: 0.7250
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5577 - accuracy: 0.7264
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7256
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7281
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7291
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7280
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7293
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5515 - accuracy: 0.7292

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_opt.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5416 - accuracy: 0.7390 - 669ms/epoch - 2ms/step
Loss: 0.5415746569633484, Accuracy: 0.7390087246894836


In [ ]:
# Export our model to HDF5 file
nn_opt.save('AlphabetSoupCharity_Optimization.h5')